In [1]:
## run groupLASSO/LASSO on the harmonic features (entropy of the spectrum power histogram)
## across all the frequencies.

## not finished yet...

import pandas as pd
import numpy as np
import os
import re
import glob

from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score

input_dir = 'adni_out04_power_histogram_and_entropy'
output_dir = 'adni_out05_lasso_all_frequencies'


In [14]:
data_files = glob.glob(input_dir + '/adni_out04_entropy_freq_*.csv')
data_files.sort()

## merge all the data_files:
all_data = []

for f in data_files:
    data = pd.read_csv(f, index_col = 0)
    data.dropna(subset=['DX'], inplace = True)
    all_data.append(data.iloc[:, 6:])
    
data_x = pd.concat([data['AGE'], data['PTGENDER'].map({'Male': 1, 'Female': 0})] + all_data, axis = 1)
data_x.columns.values[2:] = [i for i in range(264*10)]
data_x


,AGE,PTGENDER,0,1,2,3,4,5,6,7,...,2630,2631,2632,2633,2634,2635,2636,2637,2638,2639
0,84.8,1,3.398142,3.673133,3.607684,3.493571,3.404601,3.389838,3.415084,3.421256,...,3.532505,3.632422,3.570008,3.566720,3.633653,3.480775,3.456459,3.414410,3.552282,3.318798
1,76.3,0,3.530625,3.443630,3.437570,3.526242,3.585894,3.486452,3.484273,3.575364,...,3.487125,3.502944,3.563023,3.433127,3.537793,3.671293,3.520127,3.524356,3.544026,3.442273
2,89.6,0,3.050522,3.614730,3.472425,3.364719,3.171725,3.322011,3.548693,3.557887,...,3.608914,3.316018,3.440946,3.405862,3.555252,3.692156,3.468539,3.538943,3.496082,3.641701
3,65.1,0,3.074889,3.538419,3.350516,3.278443,3.497437,3.431794,3.225747,3.395888,...,3.395715,3.446629,3.413943,3.543718,3.325557,3.372889,3.171247,3.586569,3.004709,3.260015
4,57.8,1,3.537860,3.620237,3.419612,3.579573,3.601149,3.753194,3.497991,3.369718,...,3.497438,3.632419,3.529840,3.501637,3.557372,3.667401,3.369362,3.583969,3.696898,3.550346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,78.5,0,3.679113,3.519316,3.578172,3.333743,3.364221,3.163968,3.418992,3.561137,...,3.670718,3.524783,3.443845,3.468924,3.514083,3.534857,3.500693,3.633773,3.467980,3.472684
325,62.0,1,3.566919,3.408703,3.540307,3.610026,3.519628,3.438145,3.530761,3.483223,...,3.468117,3.428090,3.400476,3.575000,3.367179,3.407327,3.418302,3.457007,3.427535,3.366670
326,70.9,1,3.088003,3.575797,3.495701,3.387765,3.447946,3.495428,3.564425,3.591138,...,3.567263,3.708586,3.371401,3.144064,3.523938,3.622426,3.008386,3.141031,3.578713,3.243667
327,80.3,1,3.097272,3.538739,3.389724,3.372516,3.479099,3.532095,3.192928,3.349134,...,3.196681,3.185308,3.116343,3.514544,3.494792,3.152094,3.422050,3.334116,3.410480,3.175712


In [28]:

# comparision = [('CN', 'AD'), ('CN', 'EMCI'), ('CN', 'LMCI'), ('CN', 'SMC')]
comparision = [('CN', 'AD'), ('CN', 'EMCI'), ('CN', 'LMCI')]
result_accuracy = {comp: [] for comp in comparision}
result_coefs = {comp: [] for comp in comparision}

X = data_x.values
y = data['DX'].map({'CN': 0, 'EMCI': 1, 'LMCI': 1, 'AD': 1, 'SMC': 1}).values

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

for comp in comparision:
    # select data with ad/lmci and cn:    
    idx_ad_cn = data['DX'].isin(comp)
    # build lassoCV that tune parameters with inner cv:
    reg = LogisticRegressionCV(cv=5, random_state=0, penalty = 'l1', solver='liblinear').fit(X[idx_ad_cn,:], y[idx_ad_cn])
    # reg = LassoCV(cv=5, random_state=0, normalize = True, tol = .001).fit(X[idx_ad_cn,:], y[idx_ad_cn])
    # run outer cv:
    cv_result = cross_val_score(reg, X[idx_ad_cn,:], y[idx_ad_cn], cv=5)
    print('cross validaton result, mean %3f, std: %3f' % (cv_result.mean(), cv_result.std()))
    result_accuracy[comp] = list(cv_result)
    result_coefs[comp] = list(reg.coef_.reshape(-1))
        
    #     break
    # break

cross validaton result, mean 0.724308, std: 0.044134
cross validaton result, mean 0.513968, std: 0.021517
cross validaton result, mean 0.633548, std: 0.046540


In [54]:
## save results:
if not os.path.exists(output_dir):
    os.mkdir(output_dir)


res = pd.DataFrame(result_accuracy)
res.index = ['cv' + str(i) for i in range(1, 6)]
res.columns = ['CV vs. AD', 'CN vs. EMCI', 'CN vs. LMCI']

res.to_csv(output_dir + '/lasso_accuracy.csv')

res = pd.DataFrame(result_coefs)
res.index = ['Age', 'Sex'] + ['roi' + str(i) for i in range(2640)]
res.columns = ['CV vs. AD', 'CN vs. EMCI', 'CN vs. LMCI']

# decode roi index and frequency:
res.insert(0, 'roi_index', ['Age', 'Sex'] + [str(i%264+1) for i in range(2640)])
res.insert(0, 'frequency', ['Age', 'Sex'] + [str(i//264) for i in range(2640)])
# save coefs with non-zero coefs in at least one column.
res = res.loc[(res.iloc[:, 2:]!=0).any(axis=1)]

res.to_csv(output_dir + '/lasso_coefs.csv')


In [55]:
res

,frequency,roi_index,CV vs. AD,CN vs. EMCI,CN vs. LMCI
Age,Age,Age,0.0,-1.265163,-1.559674
Sex,Sex,Sex,0.0,0.273274,0.000000
roi1,0,2,0.0,0.000000,-0.001992
roi2,0,3,0.0,-0.301959,-0.286492
roi7,0,8,0.0,0.966632,0.000000
...,...,...,...,...,...
roi2606,9,231,0.0,0.000000,-0.050521
roi2608,9,233,0.0,-0.242174,0.000000
roi2610,9,235,0.0,0.000000,0.337706
roi2614,9,239,0.0,-0.508720,0.000000
